In [2]:
import requests

In [3]:
# WebMD Lexapro reviews
url = 'https://reviews.webmd.com/drugs/drugreview-63990-lexapro-oral'
response = requests.get(url).content

In [4]:
# make data frame to contain values from each review
# - age
# - gender
# - time on medication
# - type of user (patient, provider, etc.)
# - condition
# - overall rating
# - effectiveness rating
# - ease of use rating
# - satisfaction rating

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response, 'lxml')
reviews = soup.find('div', class_='shared-reviews-container') # get element containing all reviews